In [16]:
import sympy as sp
import tensorflow as tf
import numpy as np
import MLGeometry as mlg
from MLGeometry import bihomoNN as bnn
import cmath

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
z0, z1, z2, z3, z4 = sp.symbols('z0, z1, z2, z3, z4')
Z = [z0,z1,z2,z3,z4]
f = z0**5 + z1**5 + z2**5 + z3**5 + z4**5 + 0.5*z0*z1*z2*z3*z4

In [6]:
load_path = 'training/f0_psi0.5/300_300_300_1'
model = tf.keras.models.load_model(load_path, compile=False)

In [54]:
points_list = []
# The Bihomogeneous layer will remove the zero entries automatically.
# So here we add a small number eps to avoid being removed
eps = 0.0000001 + 0.0000001j

for k in range(100):
    point_z3 = cmath.exp(np.pi*k*2j/100)
    point_z4 = (1-point_z3**5)**(1/5)   # abs > 1, do we need this?
    points_list.append([1, 0+eps, 0+eps, point_z3, point_z4])   

In [55]:
curve = mlg.hypersurface.Hypersurface(Z, f, points=points_list, norm_coordinate=0, max_grad_coordinate=3)
curve_dataset = mlg.tf_dataset.generate_dataset(curve).batch(curve.n_points)

In [56]:
@tf.function
def cy_metric(points, Omega_Omegabar, mass, restriction):
    
    cy_metric = mlg.complex_math.complex_hessian(tf.math.real(model(points)), points)
    cy_metric_restricted = tf.matmul(restriction, tf.matmul(cy_metric, restriction, adjoint_b=True))
    
    return cy_metric_restricted

In [57]:
points, Omega_Omegabar, mass, restriction = next(iter(train_set))

In [62]:
cy_metric(points, Omega_Omegabar, mass, restriction)[5]

<tf.Tensor: shape=(3, 3), dtype=complex64, numpy=
array([[5.042442  -2.9802322e-07j, 0.25671625-1.9524626e+00j,
        0.0086575 -2.5479442e-01j],
       [0.2567169 +1.9524620e+00j, 5.329566  -6.5565109e-07j,
        0.2652911 -6.7085549e-03j],
       [0.00865772+2.5479445e-01j, 0.26529115+6.7082047e-03j,
        1.9280792 -6.7055225e-08j]], dtype=complex64)>